<a href="https://colab.research.google.com/github/prakharsrivastava/powerbi/blob/main/Pyspark__important_funtions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/chapter2/


Mounted at /content/gdrive
/content/gdrive/MyDrive/chapter2


In [3]:
!pip install findspark 
!pip install  pyspark



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=be3faf1334997cc335ed4036c191165043720181de82a33c78805708c14d97da
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [ ]:
import findspark
import pyspark
from pyspark.sql import SparkSession
sc = SparkSession.builder.master("local[1]").appName("architectinaction.com").getOrCreate()

In [ ]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10])